In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Melanoma"
cohort = "GSE244984"

# Input paths
in_trait_dir = "../../input/GEO/Melanoma"
in_cohort_dir = "../../input/GEO/Melanoma/GSE244984"

# Output paths
out_data_file = "../../output/preprocess/Melanoma/GSE244984.csv"
out_gene_data_file = "../../output/preprocess/Melanoma/gene_data/GSE244984.csv"
out_clinical_data_file = "../../output/preprocess/Melanoma/clinical_data/GSE244984.csv"
json_path = "../../output/preprocess/Melanoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Molecular patterns of resistance to immune checkpoint blockade in melanoma"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['patient: Pat10', 'patient: Pat13', 'patient: Pat14', 'patient: Pat19', 'patient: Pat21', 'patient: Pat22', 'patient: Pat23', 'patient: Pat26', 'patient: Pat27', 'patient: Pat32', 'patient: Pat39', 'patient: Pat42', 'patient: Pat44', 'patient: Pat45', 'patient: Pat46', 'patient: Pat49', 'patient: Pat5'], 1: ['resistance: CTLA4res', 'resistance: PD1res']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# Let's analyze the dataset based on the given information

# 1. Gene Expression Data Availability
# Checking for gene expression data
# Since this is a SuperSeries from GEO and there's no clear indication of what type of data it contains,
# we need to be cautious and check further. However, there's no explicit evidence that this is purely miRNA
# or methylation data, so we'll assume it might contain gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary:
# 0: Patient IDs
# 1: Resistance status (CTLA4res, PD1res) - This could be related to melanoma treatment response

# 2.1 Identify keys for trait, age, gender
trait_row = 1  # Resistance status could be considered as a trait related to melanoma
age_row = None  # Age data is not available in the provided sample characteristics
gender_row = None  # Gender data is not available in the provided sample characteristics

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert resistance status to binary trait.
    CTLA4res: resistant to CTLA4 inhibitors
    PD1res: resistant to PD1 inhibitors
    Both are related to treatment resistance in melanoma.
    """
    if not value or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip().lower()
    
    # Convert to binary: 1 for PD1 resistance, 0 for CTLA4 resistance
    if 'pd1res' in value:
        return 1
    elif 'ctla4res' in value:
        return 0
    else:
        return None

def convert_age(value: str) -> float:
    """This function is a placeholder since age data is not available."""
    return None

def convert_gender(value: str) -> int:
    """This function is a placeholder since gender data is not available."""
    return None

# 3. Save Metadata
# Perform initial filtering on usability and save information
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# If trait data is available, extract clinical features
if trait_row is not None:
    try:
        # Assuming clinical_data has been loaded in a previous step
        clinical_data = pd.read_csv(f"{in_cohort_dir}/clinical_data.csv")
        
        # Extract clinical features using the library function
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Clinical Features Preview:")
        print(preview)
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except FileNotFoundError:
        print("Clinical data file not found. Skipping clinical feature extraction.")


Clinical data file not found. Skipping clinical feature extraction.


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
Found potential subseries references:
!Series_relation = SuperSeries of: GSE244982
!Series_relation = SuperSeries of: GSE244983
!Series_relation = SuperSeries of: GSE261347

Gene data extraction result:
Number of rows: 1825
First 20 gene/probe identifiers:
Index(['A2M', 'ABCB1', 'ABCF1', 'ABL1', 'ACOT12', 'ACSF3', 'ACTA2', 'ACTB',
       'ACTR3B', 'ACVR1B', 'ACVR1C', 'ACVR2A', 'ACY1', 'ADA', 'ADAM12',
       'ADGRE1', 'ADH1A', 'ADH1B', 'ADH1C', 'ADH4'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers in the gene expression data
# Based on biomedical knowledge, these appear to be standard human gene symbols:
# - A2M (Alpha-2-Macroglobulin)
# - ABCB1 (ATP Binding Cassette Subfamily B Member 1)
# - ABCF1 (ATP Binding Cassette Subfamily F Member 1)
# - ABL1 (ABL Proto-Oncogene 1, Non-Receptor Tyrosine Kinase)
# - ACTA2 (Actin Alpha 2, Smooth Muscle)
# - ACTB (Actin Beta)
# - etc.

# These are all standard HGNC (HUGO Gene Nomenclature Committee) gene symbols
# No gene mapping is required, as these are already in the desired format

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
print(f"Top 10 gene indices before normalization: {gene_data.index[:10].tolist()}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Top 10 gene indices after normalization: {normalized_gene_data.index[:10].tolist()}")
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Extract clinical features using the clinical data from step 1
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Extract clinical features using the convert_trait function from step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # From step 2
    convert_trait=convert_trait,
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data.shape}")

# 5. Determine if the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Validate the dataset and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from juvenile myositis (JM) and childhood-onset lupus (cSLE) skin biopsies."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")

Top 10 gene indices before normalization: ['A2M', 'ABCB1', 'ABCF1', 'ABL1', 'ACOT12', 'ACSF3', 'ACTA2', 'ACTB', 'ACTR3B', 'ACVR1B']
Top 10 gene indices after normalization: ['A2M', 'ABCB1', 'ABCF1', 'ABL1', 'ACOT12', 'ACSF3', 'ACTA2', 'ACTB', 'ACTR3B', 'ACVR1B']
Shape of normalized gene data: (1820, 33)
Saved normalized gene data to ../../output/preprocess/Melanoma/gene_data/GSE244984.csv
Saved clinical data to ../../output/preprocess/Melanoma/clinical_data/GSE244984.csv
Shape of linked data: (33, 1821)


Shape of linked data after handling missing values: (33, 1821)
For the feature 'Melanoma', the least common label is '1.0' with 14 occurrences. This represents 42.42% of the dataset.
The distribution of the feature 'Melanoma' in this dataset is fine.

Saved processed linked data to ../../output/preprocess/Melanoma/GSE244984.csv
